In [ ]:
import numpy as np
import sys
sys.path.append("../")
from algorithm.koopmanlib.dictionary import PsiNN

In [ ]:
basis_function = PsiNN(layer_sizes=[100,100,100], n_psi_train=25)

In [ ]:
from duffing_oscillator import duffing_oscillator
from scipy.integrate import solve_ivp

# Parameters
delta = 0.5
alpha = -1
beta = 1

# Sample N=1000 initial points, with uniform distribution [-2, 2].
N = 1000
np.random.seed(39)
sample_data_x0 = (np.random.rand(N, 2) * 4) - 2

tspan = [0, 2.5]
nstep = 11
data = np.zeros((N, 2, nstep))

t_eval = np.linspace(tspan[0], tspan[1], nstep)

for idx, x0 in enumerate(sample_data_x0):
    sol = solve_ivp(duffing_oscillator, tspan, x0, t_eval=t_eval, args=(delta, alpha, beta))
    data[idx, 0, :] = sol.y[0]
    data[idx, 1, :] = sol.y[1]

# Pairwise shifted data X and Y
X = data[:, :, :-1] # shape: (1000, 2, 10)
Y = data[:, :, 1:]

# Transpose and reshape the data.
X = np.transpose(X, (0, 2, 1)).reshape(N * (nstep-1), 2) # shape: (10000, 2)
Y = np.transpose(Y, (0, 2, 1)).reshape(N * (nstep-1), 2)

In [ ]:
len_all = X.shape[0]
data_x_train = X[:int(0.7*len_all),:]
data_x_valid = X[int(0.7*len_all)+1:,:]

data_y_train = Y[:int(0.7*len_all),:]
data_y_valid = Y[int(0.7*len_all)+1:,:]

In [ ]:
data_train = [data_x_train, data_y_train]
data_valid = [data_x_valid, data_y_valid]

In [ ]:
## Using solver 0,3,4
from algorithm.koopmanlib.solver_0 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=50,
             batch_size=5000,
             lr=1e-4,
             log_interval=10,
             lr_decay_factor=.8)

In [ ]:
## Results from solver
evalues = solver.eigenvalues
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode()
N_dict = np.shape(evalues)[0]
# Psi_X, Psi_Y = solver.get_basis(data_train[0], data_train[1])
Koopman_matrix_K = solver.K.numpy()
Psi_X = solver.get_Psi_X().numpy()
Psi_Y = solver.get_Psi_Y().numpy()
# Assuming psi_x is your Psi_X matrix as a NumPy array
Psi_X_U, _, _ = np.linalg.svd(Psi_X/np.sqrt(Psi_X.shape[0]), full_matrices=False)
Psi_Y_U, _, _ = np.linalg.svd(Psi_Y/np.sqrt(Psi_Y.shape[0]), full_matrices=False)

# U contains the orthonormal basis vectors spanning the column space of Psi_X
print("Check if Psi_X_U is orthonormal: ", np.allclose(np.dot(Psi_X_U.T, Psi_X_U), np.eye(Psi_X_U.shape[1])))

resDMD_DL_outputs = {
    'resDMD_DL_outputs': {
        'efuns': efuns,
        'evalues': evalues,
        'kpm_modes': kpm_modes,
        'N_dict': N_dict,
        'Psi_X': Psi_X,
        'Psi_Y': Psi_Y,
        'Psi_X_U': Psi_X_U,
        'Psi_Y_U': Psi_Y_U,
        'K': Koopman_matrix_K,  # Add the Koopman matrix K to the results
    }
}

## Plot eigenvalues
import matplotlib.pyplot as plt

# Plot the unit circle for reference
theta = np.linspace(0, 2*np.pi, 100)
plt.plot(np.cos(theta), np.sin(theta), linestyle='dashed')

# Plot the real and imaginary parts of the eigenvalues
plt.scatter(evalues.real, evalues.imag, color='red', label='Eigenvalues')

# Set equal scaling for both axes
plt.axis('equal')

# Set the labels
plt.xlabel('Real part')
plt.ylabel('Imaginary part')
plt.title('Eigenvalues in the Complex Plane')

# Enable the legend
plt.legend()

# Show the plot
plt.show()

In [ ]:
## Using solver 3
from algorithm.koopmanlib.solver_3 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=50,
             batch_size=5000,
             lr=1e-4,
             log_interval=10,
             lr_decay_factor=.8)

## Results from solver
evalues = solver.eigenvalues
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode()
N_dict = np.shape(evalues)[0]
# Psi_X, Psi_Y = solver.get_basis(data_train[0], data_train[1])
Koopman_matrix_K = solver.K.numpy()
Psi_X = solver.get_Psi_X().numpy()
Psi_Y = solver.get_Psi_Y().numpy()
# Assuming psi_x is your Psi_X matrix as a NumPy array
Psi_X_U, _, _ = np.linalg.svd(Psi_X/np.sqrt(Psi_X.shape[0]), full_matrices=False)
Psi_Y_U, _, _ = np.linalg.svd(Psi_Y/np.sqrt(Psi_Y.shape[0]), full_matrices=False)

# U contains the orthonormal basis vectors spanning the column space of Psi_X
print("Check if Psi_X_U is orthonormal: ", np.allclose(np.dot(Psi_X_U.T, Psi_X_U), np.eye(Psi_X_U.shape[1])))

resDMD_DL_outputs = {
    'resDMD_DL_outputs': {
        'efuns': efuns,
        'evalues': evalues,
        'kpm_modes': kpm_modes,
        'N_dict': N_dict,
        'Psi_X': Psi_X,
        'Psi_Y': Psi_Y,
        'Psi_X_U': Psi_X_U,
        'Psi_Y_U': Psi_Y_U,
        'K': Koopman_matrix_K,  # Add the Koopman matrix K to the results
    }
}

## Plot eigenvalues
import matplotlib.pyplot as plt

# Plot the unit circle for reference
theta = np.linspace(0, 2*np.pi, 100)
plt.plot(np.cos(theta), np.sin(theta), linestyle='dashed')

# Plot the real and imaginary parts of the eigenvalues
plt.scatter(evalues.real, evalues.imag, color='red', label='Eigenvalues')

# Set equal scaling for both axes
plt.axis('equal')

# Set the labels
plt.xlabel('Real part')
plt.ylabel('Imaginary part')
plt.title('Eigenvalues in the Complex Plane')

# Enable the legend
plt.legend()

# Show the plot
plt.show()

In [ ]:
## Using solver 4
from algorithm.koopmanlib.solver_4 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=50,
             batch_size=5000,
             lr=1e-4,
             log_interval=10,
             lr_decay_factor=.8)

## Results from solver
evalues = solver.eigenvalues
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode()
N_dict = np.shape(evalues)[0]
# Psi_X, Psi_Y = solver.get_basis(data_train[0], data_train[1])
Koopman_matrix_K = solver.K.numpy()
Psi_X = solver.get_Psi_X().numpy()
Psi_Y = solver.get_Psi_Y().numpy()
# Assuming psi_x is your Psi_X matrix as a NumPy array
Psi_X_U, _, _ = np.linalg.svd(Psi_X/np.sqrt(Psi_X.shape[0]), full_matrices=False)
Psi_Y_U, _, _ = np.linalg.svd(Psi_Y/np.sqrt(Psi_Y.shape[0]), full_matrices=False)

# U contains the orthonormal basis vectors spanning the column space of Psi_X
print("Check if Psi_X_U is orthonormal: ", np.allclose(np.dot(Psi_X_U.T, Psi_X_U), np.eye(Psi_X_U.shape[1])))

resDMD_DL_outputs = {
    'resDMD_DL_outputs': {
        'efuns': efuns,
        'evalues': evalues,
        'kpm_modes': kpm_modes,
        'N_dict': N_dict,
        'Psi_X': Psi_X,
        'Psi_Y': Psi_Y,
        'Psi_X_U': Psi_X_U,
        'Psi_Y_U': Psi_Y_U,
        'K': Koopman_matrix_K,  # Add the Koopman matrix K to the results
    }
}

## Plot eigenvalues
import matplotlib.pyplot as plt

# Plot the unit circle for reference
theta = np.linspace(0, 2*np.pi, 100)
plt.plot(np.cos(theta), np.sin(theta), linestyle='dashed')

# Plot the real and imaginary parts of the eigenvalues
plt.scatter(evalues.real, evalues.imag, color='red', label='Eigenvalues')

# Set equal scaling for both axes
plt.axis('equal')

# Set the labels
plt.xlabel('Real part')
plt.ylabel('Imaginary part')
plt.title('Eigenvalues in the Complex Plane')

# Enable the legend
plt.legend()

# Show the plot
plt.show()

In [ ]:
## Using solver 1
from algorithm.koopmanlib.solver_1 import KoopmanDLSolver # resDMD solver using regression
solver = KoopmanDLSolver(dic=basis_function,
                         target_dim=np.shape(data_train)[-1],
                         reg=0.1)

solver.build(data_train=data_train,
             data_valid=data_valid,
             epochs=50,
             batch_size=5000,
             lr=1e-4,
             log_interval=10,
             lr_decay_factor=.8)

## Results from solver
evalues = solver.eigenvalues
efuns = solver.eigenfunctions(X)
kpm_modes = solver.compute_mode()
N_dict = np.shape(evalues)[0]
# Psi_X, Psi_Y = solver.get_basis(data_train[0], data_train[1])
Koopman_matrix_K = solver.K.numpy()
Psi_X = solver.get_Psi_X().numpy()
Psi_Y = solver.get_Psi_Y().numpy()
# Assuming psi_x is your Psi_X matrix as a NumPy array
Psi_X_U, _, _ = np.linalg.svd(Psi_X/np.sqrt(Psi_X.shape[0]), full_matrices=False)
Psi_Y_U, _, _ = np.linalg.svd(Psi_Y/np.sqrt(Psi_Y.shape[0]), full_matrices=False)

# U contains the orthonormal basis vectors spanning the column space of Psi_X
print("Check if Psi_X_U is orthonormal: ", np.allclose(np.dot(Psi_X_U.T, Psi_X_U), np.eye(Psi_X_U.shape[1])))

resDMD_DL_outputs = {
    'resDMD_DL_outputs': {
        'efuns': efuns,
        'evalues': evalues,
        'kpm_modes': kpm_modes,
        'N_dict': N_dict,
        'Psi_X': Psi_X,
        'Psi_Y': Psi_Y,
        'Psi_X_U': Psi_X_U,
        'Psi_Y_U': Psi_Y_U,
        'K': Koopman_matrix_K,  # Add the Koopman matrix K to the results
    }
}

## Plot eigenvalues
import matplotlib.pyplot as plt

# Plot the unit circle for reference
theta = np.linspace(0, 2*np.pi, 100)
plt.plot(np.cos(theta), np.sin(theta), linestyle='dashed')

# Plot the real and imaginary parts of the eigenvalues
plt.scatter(evalues.real, evalues.imag, color='red', label='Eigenvalues')

# Set equal scaling for both axes
plt.axis('equal')

# Set the labels
plt.xlabel('Real part')
plt.ylabel('Imaginary part')
plt.title('Eigenvalues in the Complex Plane')

# Enable the legend
plt.legend()

# Show the plot
plt.show()